In [1]:
import os
import pandas as pd
import numpy as np
import time
import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
os.listdir(FOLDER)

['mulliken_charges.csv',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'train.csv',
 'test.csv',
 'dipole_moments.csv',
 'potential_energy.csv',
 'structures.csv',
 'out',
 'scalar_coupling_contributions.csv',
 'dsgdb9nsd_000001.xyz',
 'structures.zip']

In [ ]:
from process import gen_pairs_list, get_cos_3J

In [3]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
df = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

In [4]:
df_structures_idx = df_structures.set_index('molecule_name')
df_idx = df.set_index('molecule_name')
mols = df['molecule_name'].unique()
types_3J = ['3JHH', '3JHC', '3JHN']

In [8]:
col_cos = np.zeros([0,2])
id_arr = np.zeros(0)
start = time.time()
types_3J = ['3JHH', '3JHC', '3JHN']
for m in mols:
    df_tr = df_idx.loc[m]
    cos_arr = np.zeros([df_tr.shape[0]])
    for t in types_3J:
        pairs_list = gen_pairs_list(df_idx, df_structures_idx, m, t)
        if len(pairs_list) == 0:
            continue
        for pair in pairs_list:
            cos = get_cos_3J(df_structures_idx, m, pair)
            col_cos = np.vstack([col_cos, cos])
            id_arr = np.hstack([id_arr, pair[4]])

se_id = pd.Series(id_arr, name='id', dtype='int32')
df_cos = pd.DataFrame(col_cos, columns=['cos_3j', 'cos_3j^2']).astype('float32')
df_cos_3j = pd.concat([se_id, df_cos], axis=1)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

/home/ryoji/Documents/git/kaggle/ml/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


KeyboardInterrupt: 

In [ ]:
df_cos_3j.to_pickle(OUTPUT + '20190629_cos_3j_2.pickle', index=False)